# Scikit-learn - 机器学习算法库

Scikit-learn是Python中最流行的机器学习库，提供了丰富的机器学习算法。

## 本教程内容
- 使用支持向量机(SVM)进行MNIST手写数字分类
- 数据预处理和特征工程
- 模型训练、验证和测试
- 性能评估和可视化

In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("开始加载MNIST数据集...")

# 加载MNIST数据集（sklearn内置的简化版本）
digits = datasets.load_digits()
X, y = digits.data, digits.target

print(f"数据集形状: {X.shape}")
print(f"标签形状: {y.shape}")
print(f"类别数量: {len(np.unique(y))}")
print(f"特征数量: {X.shape[1]}")

# 显示一些示例图像
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap='gray')
    ax.set_title(f'标签: {digits.target[i]}')
    ax.axis('off')
plt.tight_layout()
plt.show()